In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib.ipynb

# 修正したモデルから卒論時に集計したデータを作成する

1. 表

| ベンチマーク名 | 平均誤差率(%) | コスト比(%) |
|---------|----------|---------|


2. 表

| ベンチマーク名 | 採用割合(最大MAPE(%), 最小MAPE(%)) |
|---------|----------------------------|
|         | モデル(1), モデル(2), ...        |


* 平均誤差率：大規模実行時の関数コール回数との比較
* MAPE：トレーニングデータとの比較

In [ ]:
plt.figure(figsize=(4, 3))
plt.xlabel("使用したプロファイル数")
plt.ylabel("平均誤差率(%)")

In [ ]:
# pd.get_option("display.max_columns")
# pd.get_option("display.max_rows")
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [ ]:
# ベンチマーク名・関数名・プロセス数・問題サイズを指定することで、その条件での関数コール回数を取得する関数

def returnSpecificData(benchmarkName="cg", functionName=".TAU_application", process=256, benchmarkClass="D"):
    targetRawDF = returnRawDF(Benchmark=benchmarkName, functionName=functionName, benchmarkClass=[
                              benchmarkClass], FixedProcess=process, Processes=[process], FixedBenchmarkClass=benchmarkClass)
    return targetRawDF.iat[0, 0]
# returnSpecificData(benchmarkName="mg", functionName="BUBBLE", process=256, benchmarkClass="B")

In [ ]:
# benchmarksからbt, spを除外する
benchmarks = [benchmark for benchmark in benchmarks if benchmark !=
              'bt' and benchmark != 'sp']
# pandasのDFをprintした時の幅を広げる
pd.set_option('display.width', 100)

In [ ]:
dictTmp = returnDictForPlotPerNumOfUsedData(Benchmark=benchmarks, fix="Class", benchmarkClass=[
    "A", "B", "C", "D"], FixedProcess=64, Processes=[1, 2, 4, 8, 16, 32, 64, 128, 256], FixedBenchmarkClass="C")

In [ ]:
pd.options.display.float_format = '{:.4g}'.format

tmpDF = pd.DataFrame()
for benchmark in benchmarks:
    listToLearn = [1, 2, 4, 8, 16, 32, 64, 128]
    listToPredict = [256]
    benchmark_x = dictTmp[benchmark]["x"]
    benchmark_y = dictTmp[benchmark]["y"]
    index = benchmark_x.index(len(listToLearn))
    MAPE = benchmark_y[index]
    relativeCost = returnRelativeCost(benchmark=benchmark, variablesToLearn=listToLearn,
                                      variablesToPredict=listToPredict, fixedClassOrProcess="Class", fixed="C")
    dictRowData = {"ベンチマーク名": benchmark.upper(
    ), "平均絶対誤差率[％]": MAPE, "相対コスト[％]": relativeCost}
    iDF = pd.DataFrame.from_dict(dictRowData, orient='index').T
    tmpDF = tmpDF.append(iDF)
tmpDFMean = tmpDF.mean()
type(tmpDFMean)
print(tmpDF.to_latex(index=False))

In [ ]:
# dictTmp

plt.figure(figsize=(5.72, 4), dpi=200)
for benchmark in list(dictTmp.keys()):
    x = dictTmp[benchmark]["x"]
    y = dictTmp[benchmark]["y"]
    plt.plot(x, y, marker='o', label=benchmark.upper())
    plt.legend()
    plt.xlabel("使用したプロファイル数")
    plt.ylabel("平均絶対誤差率[％]")

In [ ]:
plt.figure(figsize=(5.72, 4), dpi=200)

# Extra-PでfixProcessデータを入力して出力したモデルの図時
plot_x = np.linspace(0.8, 256, 500)
# -3590464.6990329633 + 3759195.349891038 * p^(1/4)
plot_y = []
for x in plot_x:
    plot_y.append(2286768.3333333326 + 301997.61904761934 * math.log2(x)**(1))
plt.plot(plot_x, plot_y, label="ExtraP")

x = [1, 2, 4, 8, 16, 32, 64, 128]
y = [1984770.0, 2263540.0, 2821070.0, 3936140.0,
     3936140.0, 3936140.0, 3936140.0, 3936140.0]
x = np.array(x).reshape(-1, 1)
y = np.array(y).reshape(-1, 1)
plt.scatter(x, y, marker="o", label="予測に用いた関数コール回数")
plot_x = np.array(plot_x).reshape(-1, 1)
x_target = [256]
y_target = [3936140]
plt.scatter(x_target, y_target, marker="o", label="予測したい関数コール回数の実測値")

benchmarkName = "CG"
functionName = "ICNVRT"

# 線形モデル
# 対数モデル

# 反比例モデル
modelIpMk2 = ModelIp_mk2(train_x=x, train_y=y, target_x=x_target, target_y=y_target,
                         benchmark_name=benchmarkName, function_name=functionName)
modelIpMk2.calc_lr()
plot_y_IpMk2 = modelIpMk2.predict(plot_x)
plt.plot(plot_x, plot_y_IpMk2, label="反比例モデル")
# 線形飽和モデル
modelBranchMk2 = ModelBranch_mk2(train_x=x, train_y=y, target_x=x_target,
                                 target_y=y_target, benchmark_name=benchmarkName, function_name=functionName)
modelBranchMk2.calc_lr()
plot_y_BranchMk2 = modelBranchMk2.predict(plot_x)
plt.plot(plot_x, plot_y_BranchMk2, label="線形飽和モデル")
# # 線形モデル
# model_lin = ModelLin(x, y, "CG", "ICNVRT", test_ratio=0)
# model_lin.calc_lr()
# plot_y_lin = model_lin.predict(plot_x)
# plt.plot(plot_x, plot_y_lin, label="線形モデル")
# # 対数モデル
# model_log10 = ModelLog10(x, y, "CG", "ICNVRT", test_ratio=0)
# model_log10.calc_lr()
# plot_y_log10 = model_log10.predict(plot_x)
# plt.plot(plot_x, plot_y_log10, label="対数モデル")
# # 反比例モデル
# model_ip = ModelIP(x, y, "CG", "ICNVRT", test_ratio=0)
# model_ip.calc_lr()
# plot_y_ip = model_ip.predict(plot_x)
# plt.plot(plot_x, plot_y_ip, label="反比例モデル")
# # 線形飽和モデル
# model_branch = ModelBranch(x, y, "CG", "ICNVRT", test_ratio=0)
# model_branch.calc_lr()
# plot_y_branch = model_branch.predict(plot_x)
# plt.plot(plot_x, plot_y_branch, label="線形飽和モデル")
# 凡例の表示
plt.legend()
# 軸ラベルの設定
plt.ylabel("関数コール回数")
plt.xlabel("実行コア数")

plt.scatter(x, y, marker="o")

In [ ]:
# 実際にプロットする


# print(f"fix={fix}, benchmarkClasses={benchmarkClasses}, fixedProcess={fixedProcess}, Processes={processes}, FixedBenchmarkClass={fixedBenchmarkClass}")
# print(f"targetNumOfProcess={targetNumOfProcess}, targetProblemSize={fixedBenchmarkClass}, fix={fix}")

# DF = returnRawDFperBenchmark(Benchmark="mg", fix="Process", benchmarkClass=["A", "B", "C", "D"], Processes=[
#                              1, 2, 4, 8, 16, 32, 64, 128, 256], FixedBenchmarkClass="B", FixedProcess=64)
# DF.dropna(how='any')
# DF

In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

In [ ]:
benchmarkNamesExcludeBTSP = ["cg", "ep", "ft", "is", "lu", "mg"]
# classes = ["A", "B", "C", "D"]
classes = ["B"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]
targetIndex = -1
csvDirPath = "./csv_files/"

dfByDatumExcludeBTSP = returnDFSummarizedData(
    benchmarkNames=benchmarkNamesExcludeBTSP, classes=classes, processes=processes, targetIndex=targetIndex, csvDirPath=csvDirPath)
# dfByDatumExcludeBTSP

dictForLatexTable = {}
numOfData = 0
for benchmarkName in benchmarkNamesExcludeBTSP:
    dictForLatexTable[benchmarkName] = dfByDatumExcludeBTSP[dfByDatumExcludeBTSP["benchmarkName"] == benchmarkName]
    numOfData += len(
        dfByDatumExcludeBTSP[dfByDatumExcludeBTSP["benchmarkName"] == benchmarkName])

numOfData

In [ ]:


listForDF = []

for benchmarkName in benchmarkNamesExcludeBTSP:
    listForDF.append(returnSeriesOfDatumPerBenchmark(
        inputDF=dictForLatexTable[benchmarkName]))
DF = pd.DataFrame(listForDF)
print(DF.to_latex(index=False))

In [ ]:
test_returnSeriesOfDatumPerBenchmark()

In [ ]:
resultIs = dictForLatexTable["is"]
# resultIs
resultIsAtModelBranch = resultIs[resultIs["objectBestModelName"]
                                 == "ModelBranch"]
datumX = resultIsAtModelBranch["usedDataX"].tolist()
datumY = resultIsAtModelBranch["usedDataY"].tolist()

datumX
datumY

# returnSeriesOfData(benchmarkName="is", functionName="double_randlc(double_*_double_*)", rawX=dataX, rawY=dataY, fixProcessOrClass="Class", fixed="B", targetProcess=256, targetBenchmarkClass="B", targetFunctionCallNum=-1, csvDirPath="./csv_files")

In [ ]:
resultIs = dictForLatexTable["ft"]
# resultIs
resultIsAtModelBranch = resultIs[resultIs["objectBestModelName"]
                                 == "ModelBranch"]
resultIsAtModelBranchOfNotLowMAPE = resultIsAtModelBranch[
    resultIsAtModelBranch["MAPEOfBestModel"] > 1]
resultIsAtModelBranchOfNotLowMAPE
datumX = resultIsAtModelBranchOfNotLowMAPE["usedDataX"].tolist()
datumY = resultIsAtModelBranchOfNotLowMAPE["usedDataY"].tolist()

datumX
datumY

for dataIndex in range(len(datumX)):
    plt.figure()
    plt.scatter(datumX[dataIndex], datumY[dataIndex])

# returnSeriesOfData(benchmarkName="is", functionName="double_randlc(double_*_double_*)", rawX=dataX, rawY=dataY, fixProcessOrClass="Class", fixed="B", targetProcess=256, targetBenchmarkClass="B", targetFunctionCallNum=-1, csvDirPath="./csv_files")

In [ ]:
# 生データの取得
cgDF = returnCollectedExistingData(benchmarkNames=["cg"], classes=["A", "B", "C", "D"], processes=[
                                   1, 2, 4, 8, 16, 32, 64, 128, 256], csvDirPath="./csv_files/")
cgDF
# ベンチマーククラスがAの情報を取得
cgDFfixedA = cgDF[cgDF["benchmarkClass"] == "A"]
cgDFfixedA
# 関数名のリストを取得
functionNames = sorted(list(set(cgDFfixedA["functionName"])))
print(functionNames)

# 関数名を関数名のリストから抽出
functionNameCG = cgDFfixedA[cgDFfixedA["functionName"] == "CG"]
functionNameCG

# 説明変数と目的変数とをリスト化したものを抽出
# プロセス数
raw_x = functionNameCG['process'].tolist()
# 関数コール回数
raw_y = functionNameCG['functionCallNum'].tolist()

print(f"raw_x={raw_x}")
print(f"raw_y={raw_y}")

bencmarkName = "CG"
functionName = "CG"
fixProcessOrClass = "Class"
fixed = "A"
targetProcess = 256
targetBenchmarkClass = fixed
targetFunctionCallNum = raw_y[-1]
returnSeriesOfData(benchmarkName="benhmarkName", functionName="functionName", rawX=[1, 2, 3], rawY=[
                   1, 2, 3], fixProcessOrClass="Class", fixed="B", targetProcess=256, targetBenchmarkClass="B", targetFunctionCallNum=-1, csvDirPath="./csv_files/")

In [ ]:
%reset

In [ ]:
# ノートブック中で変数のみを記述することでデータフレームをきれいに表示させる設定の有効化
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 問題サイズD, コア数256での関数コール回数を予測する

In [ ]:
# TODO：BT, SP以外のベンチマーク名を入れる
benchmarkNames = ['cg', 'ep', 'ft', 'is', 'lu', 'mg']
classes = ["A", "B", "C", "D"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]

dictForSummarizedResult = {}
columnName = ["benchmarkName", "functionName", "score", "relativeErrorRate"]
dfForSummarizedResult = pd.DataFrame(columns=columnName)
for benchmarkName in benchmarkNames:
    # ベンチマークごとにscoreを保持するためのリスト
    listForSummarizedResultPerBenchmarkName = []
    # 学習用生データ
    DF = returnCollectedExistingData(benchmarkNames=[
                                     benchmarkName], classes=classes, processes=processes, csvDirPath="./csv_files/")
    # 重複のない関数名のリスト
    functionNames = list(set(DF["functionName"]))
    usefulFunctionNames = []
    # このループで関数ごとのデータが問題サイズパターン数xコア数パターン数 分だけ存在する関数名のリストを作成する
    for functionName in functionNames:
        # 関数ごとに生データを集計
        dfPerFunction = DF[DF["functionName"] == functionName]
        if len(classes) * len(processes) == len(dfPerFunction):
            usefulFunctionNames.append(functionName)
    if len(usefulFunctionNames) == 0:
        continue
    # 関数ごとのデータを抽出
    for functionName in usefulFunctionNames:
        # 問題サイズを数値化したカラムを追加
        listBenchmarkClass = DF["benchmarkClass"].tolist()
        DFWithNumInBenchmarkClass = DF.assign(
            benchmarkClassInNum=convertBenchmarkClasses_problemSizeInNPB(listBenchmarkClass))
        # 学習用データ
        dfPerFunctionForTrain = DFWithNumInBenchmarkClass[(
            DFWithNumInBenchmarkClass["functionName"] == functionName)]
        dfPerFunctionForTest = DFWithNumInBenchmarkClass[(DFWithNumInBenchmarkClass["functionName"] == functionName) & (
            DFWithNumInBenchmarkClass["benchmarkClass"] == "D") & (DFWithNumInBenchmarkClass["process"] == 256)]

        # x:説明変数, t:目的変数
        trainX = dfPerFunctionForTrain[["process", "benchmarkClassInNum"]]
        trainT = dfPerFunctionForTrain[["functionCallNum"]]
        testX = dfPerFunctionForTest[["process", "benchmarkClassInNum"]]
        testT = dfPerFunctionForTest[["functionCallNum"]]
        # 重回帰分析する
        reg_model = LinearRegression()
        reg_model.fit(trainX, trainT)
        # 関数ごとの結果をベンチマークごとの結果に入れる
        scorePerFunction = reg_model.score(trainX, trainT)
        listForSummarizedResultPerBenchmarkName.append(scorePerFunction)
        # 予測を実施して、相対誤差を算出
        predictedTByTestX = reg_model.predict(testX)
        predictedData = predictedTByTestX[0][0]
        realData = testT["functionCallNum"].tolist()[0]
        relativeErrorPerFunction = abs(predictedData - realData)/realData * 100
        ##
        dfPerFunction = pd.DataFrame(index=columnName, data=[
                                     benchmarkName, functionName, scorePerFunction, relativeErrorPerFunction]).T
        dfForSummarizedResult = dfForSummarizedResult.append(dfPerFunction)

# ( A ~ D ) * (1 ~ 256) のすべての条件を
# 満たしていたら、リストに追加
# 満たしていなければ、なにもしない

In [ ]:
dfForSummarizedResult

In [ ]:
inputDF = dfForSummarizedResult


benchmarkNamesInDF = list(set(dfForSummarizedResult["benchmarkName"].tolist()))

listForLatexTable = []
for benchmarkName in benchmarkNamesInDF:
    print(benchmarkName)
    inputDFPerBenchmark = inputDF[inputDF["benchmarkName"] == benchmarkName]
    meanData = inputDFPerBenchmark.mean()
    print(type(meanData))
    meanData["benchmarkName"] = f"{benchmarkName.upper()}({len(inputDFPerBenchmark)})"
    listForLatexTable.append(meanData)
DF = pd.DataFrame(listForLatexTable)

DF = DF.sort_index(axis='columns')
DF
# relativeErrorの単位は[%]ではない。scoreの値はscore()で取得できたもの
DF.columns = ["ベンチマーク名(関数の個数)", "MAPE(予測対象関数コール回数に対する)", "決定係数"]
print(DF.to_latex(index=False))

In [ ]:
forInputDF = returnDFSummarizedData(
    benchmarkNames=["cg", "ep", "ft", "is", "lu", "mg"],
    classes=["C"],
    processes=[2, 4, 8, 16, 32, 64, 128, 256],
    targetIndex=-1,
    csvDirPath="./csv_files/",
)

benchmarkNames = list(set(forInputDF["benchmarkName"].tolist()))
columnsNames = ["ベンチマーク名(関数の個数)", "MAPE(予測対象関数コール回数に対する)"]
listForRelativeErrorTable = []
for benchmarkName in benchmarkNames:
    forInputDFPerBenchmark = forInputDF[forInputDF["benchmarkName"]
                                        == benchmarkName]
    column1 = f"{benchmarkName.upper()}({len(forInputDFPerBenchmark)})"
    seriesOfMean = forInputDFPerBenchmark.mean()
    seriesOfMeanRelativeErrorRate = seriesOfMean["RelativeErrorRate"]
    column2 = int(seriesOfMeanRelativeErrorRate * 100) / 100
    listForRelativeErrorTable.append([{column1}, {column2}])
print(pd.DataFrame(listForRelativeErrorTable,
      columns=columnsNames).to_latex(index=False))

# forInputDFPerBenchmark

In [ ]:
pd.DataFrame(listForRelativeErrorTable, columns=columnsNames)

In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

In [ ]:
benchmarkNames = ['cg', 'ep', 'ft', 'is', 'lu', 'mg']

classes = ["A", "B", "C", "D"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]
targetClass = classes[-1]
targetProcess = processes[-1]

# 学習用生データ
DF = returnCollectedExistingData(benchmarkNames=benchmarkNames,
                                 classes=classes, processes=processes, csvDirPath="./csv_files/")
DFByValidFunction = returnDFwithFunctionsExecUnderAllConditions(
    inputDF=DF, classes=classes, processes=processes)
# 問題サイズを数値化したカラムを追加
listBenchmarkClass = DFByValidFunction["benchmarkClass"].tolist()
# 生データにカラムなどを加えた整形済みのDF
shapedDF = DFByValidFunction.assign(
    benchmarkClassInNum=convertBenchmarkClasses_problemSizeInNPB(listBenchmarkClass))

# 説明変数のカラム名のリスト
expVarColNames = ["process", "benchmarkClassInNum"]
# 目的変数のカラム名のリスト
resVarColNames = ["functionCallNum"]

# データ内にあるベンチマーク名のリスト
benchmarkNames = set(shapedDF["benchmarkName"].tolist())

# 集計前のデータを作成
# DFで[functionName | benchmarkName | expVarDatumDict | resVarDatumDict | modelsName | dictAggregateResult]がカラム名

dictToMakeSummary = {}
for benchmarkName in benchmarkNames:
    DFperBenchmark = shapedDF[shapedDF["benchmarkName"] == benchmarkName]
    # すべての条件で実行された関数名のリスト
    validFunctionNames = list(
        set(DFperBenchmark["functionName"].tolist()))
    listToMakeDF = []
    print(f"benchmakName={benchmarkName}, 関数の個数:{len(validFunctionNames)}")
    for validFunctionName in validFunctionNames:
        # 3モデルを一気に作成するmodels()を利用
        inputDFperFunction = DFperBenchmark[(DFperBenchmark["functionName"] == validFunctionName) & (
            DFperBenchmark["benchmarkName"] == benchmarkName)].reset_index()
        targetDFperFunction = inputDFperFunction[(inputDFperFunction["benchmarkClass"] == targetClass) & (
            inputDFperFunction["process"] == targetProcess)]
        # 説明変数のカラム名リストを作成
        expVarColNames = ["process", "benchmarkClassInNum"]
        # 目的変数のカラム名リストを作成
        resVarColNames = ["functionCallNum"]
        # モデルを一括で作成
        if len(targetDFperFunction) == 0:
            print(
                f"benchmarkName={benchmarkName}, functionName={validFunctionName}")
            continue
        returnedDF = returnDFtoMakeSummary(inputDF=inputDFperFunction, benchmarkName=benchmarkName, validFunctionName=validFunctionName,
                                           targetClass=targetClass, targetProcess=targetProcess, expVarColNames=expVarColNames, resVarColNames=resVarColNames)
        listToMakeDF.append(returnedDF)
    if len(listToMakeDF) == 0:
        continue
    inputDFtoMakeDFperBenchmark = pd.concat(
        listToMakeDF).reset_index(drop=True)
    returnedDict = convertDictToMakeSummary(
        inputDF=inputDFtoMakeDFperBenchmark, modelAdoptionRate=True, averageRelativeError=True)
    dictToMakeSummary[benchmarkName] = returnedDict
# TODO:作成したDFを入力として集計関数を実行

modelAdoptionRate = returnedDict["modelAdoptionRate"]
averageRelativeError = returnedDict["averageRelativeError"]
returnedDict

modelAdoptionRate

averageRelativeError

In [ ]:
columnBenchmarkName = []
# 採用割合
columnAdoptionRateLog = []
columnAdoptionRateIp = []
columnAdoptionRateLin = []
# 平均相対誤差率
columnAverageRelativeError = []

for benchmarkName in dictToMakeSummary.keys():
    # 関数の個数
    numOfFunctions = 0

    # 採用割合
    modelAdoptionRate = dictToMakeSummary[benchmarkName]["modelAdoptionRate"]
    modelNames = list(modelAdoptionRate.keys())
    for modelName in modelNames:
        #         print(modelName)
        numOfFunctions += modelAdoptionRate[modelName]["count"]
    dictToMakeSummary[benchmarkName]["関数の個数"] = numOfFunctions

    # 採用割合を算出するモデルが３つであることを仮定している
    # modelLog
    adoptionRateLog = int(
        modelAdoptionRate["modelLog"]["count"]/numOfFunctions * 100)
    if modelAdoptionRate["modelLog"]["count"] == 0:
        logMinMAPE = "-"
        logMaxMAPE = "-"
    else:
        logMinMAPE = int(float(modelAdoptionRate["modelLog"]["min"])*10)/10
        logMaxMAPE = int(float(modelAdoptionRate["modelLog"]["max"])*10)/10
    # modelIp
    adoptionRateIp = int(
        modelAdoptionRate["modelIp"]["count"]/numOfFunctions * 100)
    if modelAdoptionRate["modelIp"]["count"] == 0:
        ipMinMAPE = "-"
        ipMaxMAPE = "-"
    else:
        ipMinMAPE = int(float(modelAdoptionRate["modelIp"]["min"])*10)/10
        ipMaxMAPE = int(float(modelAdoptionRate["modelIp"]["max"])*10)/10
    # modelLin
    adoptionRateLin = 100 - adoptionRateIp - adoptionRateLog
    if modelAdoptionRate["modelLin"]["count"] == 0:
        linMinMAPE = "-"
        linMaxMAPE = "-"
    else:
        linMinMAPE = int(float(modelAdoptionRate["modelLin"]["min"])*10)/10
        linMaxMAPE = int(float(modelAdoptionRate["modelLin"]["max"])*10)/10

    # Latex化するためにカラムとして入れる
    columnBenchmarkName.append(f"{benchmarkName.upper()}({numOfFunctions})")
    columnAdoptionRateLog.append(
        f"{adoptionRateLog}({logMinMAPE},{logMaxMAPE})")
    columnAdoptionRateIp.append(f"{adoptionRateIp}({ipMinMAPE},{ipMaxMAPE})")
    columnAdoptionRateLin.append(
        f"{adoptionRateLin}({linMinMAPE},{linMaxMAPE})")

    # 相対誤差率
    averageRelativeError = dictToMakeSummary[benchmarkName]["averageRelativeError"]
    columnAverageRelativeError.append(int(averageRelativeError*100)/100)

In [ ]:
dictToMakeSummary

columnBenchmarkName

columnAdoptionRateLog

columnAdoptionRateIp

columnAdoptionRateLin

columnAverageRelativeError

# 採用割合
採用割合 = pd.DataFrame({"ベンチマーク名": columnBenchmarkName,
                    "反比例モデル": columnAdoptionRateIp, "対数モデル": columnAdoptionRateLog, "線形モデル": columnAdoptionRateLin})
# 相対誤差率
相対誤差率 = pd.DataFrame({"ベンチマーク名": columnBenchmarkName,
                     "相対誤差率[%]": columnAverageRelativeError})

print(採用割合.to_latex(index=False))

print(相対誤差率.to_latex(index=False))

In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

class ModelBranchForMultipleRegression(ModelBaseForMultipleRegression):
    # 線形モデル（重回帰分析）

    def transformDataForModel(self, inputDF):
        # inputDFで与えられたデータをモデルに適した形に変形する
        return(inputDF)

    def setUpDataBeforeCalcLr(self):
        # 説明変数・目的変数を変換する関数
        # モデル構築用データ
        self.dataXForPredict = self.transformDataForModel(
            self.rawExplanaoryVariable)
        self.dataTForPredict = self.transformDataForModel(
            self.rawResponseVariable)
        # テスト用データ
        self.dataXForTest = self.transformDataForModel(
            self.rawExplanaoryVariableForTest)
        self.dataTForTest = self.transformDataForModel(
            self.rawResponseVariableForTest)

    def calcLr(self):
        # 実際にモデルを構築する
        self.lr = LinearRegression()
        self.lr.fit(self.dataXForPredict, self.dataTForPredict)

    def predict(self, inputDF):
        # inputDFのデータから構築されたモデルを使って予測を行う

        # inputDFから説明変数データのみを取得
        inputDFOnlyExplanatoryVariableColumn = inputDF[self.explanatoryVariableColumnNames]
        # 予測を実行
        result = self.lr.predict(inputDFOnlyExplanatoryVariableColumn)

        return(result)


def test_ModelBranchForMultipleRegression():
    # 単なる線形モデル
    # 説明変数
    plotX = np.linspace(0, 20, 10)
    plotY = np.linspace(20, 40, 10)
    plotZ = np.linspace(40, 60, 10)
    # 目的変数
    plotT = plotX + 2 * plotY + 3 * plotZ + 4

    # DFを作成する
    # カラム名のリスト
    columnNames = ["plotX", "plotY", "plotZ", "plotT"]
    datumForDF = [plotX, plotY, plotZ, plotT]
    inputDFForTest = pd.DataFrame(index=columnNames, data=datumForDF).T
    inputDFForTest["functionName"] = "functionName"

    # 目的変数・説明変数のカラム名のリスト
    # 目的変数のカラム名のリスト
    columnNamesForExp = columnNames[:-1]
    # 説明変数のカラム名のリスト
    columnNamesForRes = columnNames[-1:]

    # 予測をする
    # モデルオブジェクトの作成
    objectModel = ModelBranchForMultipleRegression(inputDF=inputDFForTest, explanatoryVariableColumnNames=columnNamesForExp,
                                                   responseVariableColumnNames=columnNamesForRes, conditionDictForTest={})
    # モデルの生成の準備
    objectModel.setUpDataBeforeCalcLr()
    # モデルの生成
    objectModel.calcLr()
    # モデルによる予測
    # 入力データDFを作成
    inputDFForPredict = pd.DataFrame(inputDFForTest.tail(1))
    predictedNum = objectModel.predict(inputDFForPredict)

    # 相対誤差率でテスト対象のデータが想定通りに動作しているかを判断する
    # 相対誤差率を計算するために実データを取得する
    realNum = plotT[-1]
    relativeErrorRate = returnRelativeErrorRate(
        realNum=realNum, predictedNum=predictedNum)

    assert relativeErrorRate < 1

    # 線形飽和モデル
    branchIndex = 5
    # 説明変数
    branchX = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    branchY = [11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
    branchZ = [21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
    # 目的変数
    branchT_X = returnListForBranchModel(
        inputList=branchX, branchIndex=branchIndex, a=1, b=2)
    branchT_Y = returnListForBranchModel(
        inputList=branchY, branchIndex=branchIndex, a=3, b=4)
    branchT_Z = returnListForBranchModel(
        inputList=branchZ, branchIndex=branchIndex, a=5, b=6)
    branchT = []
    for numX, numY, numZ in zip(branchT_X, branchT_Y, branchT_Z):
        branchT.append(numX+numY+numZ)
    # DFを作成する
    # カラム名のリスト
    inputDFForTest = pd.DataFrame(
        {"branchX": branchX, "branchY": branchY, "branchZ": branchZ, "branchT": branchT})
    # 目的変数のカラム名のリスト
    expVarName = ["branchX", "branchY", "branchZ"]
    # 説明変数のカラム名のリスト
    resVarName = ["branchT"]
    # 関数名
    inputDFForTest["functionName"] = "functionName"
    # 予測のためのモデルオブジェクトの作成
    objectModel = ModelBranchForMultipleRegression(
        inputDF=inputDFForTest, explanatoryVariableColumnNames=expVarName, responseVariableColumnNames=resVarName, conditionDictForTest={})
    # モデルの生成の準備
    objectModel.setUpDataBeforeCalcLr()
    objectModel.calcLr()
    # モデルによる予測
    inputDFForPredict = pd.DataFrame(inputDFForTest.tail(1))
    predictedNum = objectModel.predict(inputDFForPredict)
    # 相対誤差率でテスト対象のデータに焚いてモデルが想定通りに動作しているかを判断する
    realNum = branchT[-1]
    relativeErrorRate = returnRelativeErrorRate(
        realNum=realNum, predictedNum=predictedNum)
    print(f"relatieErrorRate={relativeErrorRate}")
    assert relativeErrorRate < 1


test_ModelBranchForMultipleRegression()